# Detectron2 Beginner's Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

Welcome to detectron2! This is the official colab tutorial of detectron2. Here, we will go through some basics usage of detectron2, including the following:
* Run inference on images or videos, with an existing detectron2 model
* Train a detectron2 model on a new dataset

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.


# Install detectron2

In [ ]:
# # install dependencies: 
# # !sudo apt-get update -yq && DEBIAN_FRONTEND=noninteractive apt-get install -y git vlc python-opencv curl unzip python-tk wget vlc libgl1-mesa-glx 
# !pip install pyyaml>=5.1 torch torchvision torchaudio==0.7.0 opencv-python  av
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

# import torch, torchvision                                                                                                              
# print(torch.__version__, torch.cuda.is_available())
# !gcc --version
# # opencv is pre-installed on colab

In [ ]:
# cfg = get_cfg()
# # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# predictor = DefaultPredictor(cfg)
# outputs = predictor(im)

In [ ]:
# # install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# # See https://detectron2.readthedocs.io/tutorials/install.html for instructions
# import torch
# assert torch.__version__.startswith("1.7")
# # exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# !mkdir -p dataset
# %cd dataset
# !curl -L "https://app.roboflow.com/ds/0B7BgH0Prx?key=AfYsS8tajq" > roboflow.zip; unzip -o roboflow.zip > /dev/null; rm roboflow.zip
# %cd -

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
import torch

setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets.coco import load_coco_json
DatasetCatalog.clear()
DatasetCatalog.register("beers_test", lambda: load_coco_json("dataset/test/_annotations.coco.json", "dataset/test"))
DatasetCatalog.register("beers_train", lambda: load_coco_json("dataset/train/_annotations.coco.json", "dataset/train"))
DatasetCatalog.register("beers_valid", lambda: load_coco_json("dataset/valid/_annotations.coco.json", "dataset/valid"))

In [ ]:
classes = ["Geladeira", "aguacrystal200ml", "delvallegoiaba290ml","delvallemaracuja290ml","delvalleuva290ml", "leaoguarana300ml","leaoguaranaacai300ml", "monsterenergy473ml", "pratslaranja900ml","toddynho150ml"]
MetadataCatalog.get("beers_train").set(thing_classes= classes)
MetadataCatalog.get("beers_test").set(thing_classes= classes)
MetadataCatalog.get("beers_valid").set(thing_classes= classes)

In [ ]:
# Show the object labels

In [ ]:
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode

dataset_dicts = DatasetCatalog.get("beers_valid")
meta_valid = MetadataCatalog.get("beers_valid")
# plt.style.use("dark_background")
# fm = plt.get_current_fig_manager()
# fm.toolbar.pan()


for d in random.sample(dataset_dicts, 15):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=meta_valid, scale=0.8, instance_mode=ColorMode.IMAGE_BW)

    vis = visualizer.draw_dataset_dict(d)
    test_img = vis.get_image()[:, :, ::-1]

    plt.figure(figsize=(15,15))
    plt.imshow(test_img)
    plt.show() 

## Train!

Now, let's fine-tune a COCO-pretrained R50-FPN Mask R-CNN model on the balloon dataset. It takes ~6 minutes to train 300 iterations on Colab's K80 GPU, or ~2 minutes on a P100 GPU.


In [ ]:
    import torch
    torch.cuda.empty_cache()


In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/rpn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("beers_train", "beers_test")
cfg.DATASETS.TEST = () #"beers_test",)
cfg.TEST.EVAL_PERIOD = 50
cfg.DATALOADER.NUM_WORKERS = 4
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/rpn_R_50_FPN_1x.yaml")  # Let training initialize from model zoo
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl"
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000    #adjust up if val mAP is still rising, adjust down if overfit
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg)                                           
trainer.resume_or_load(resume=False)

In [ ]:
# # Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
trainer.train()

In [ ]:
# Evaluate its performance using AP metric implemented in COCO API.


In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
#print("cfg: ", cfg)
evaluator = COCOEvaluator("beers_valid", cfg, False, output_dir="./output/")
#print("Eval: ", eval)
val_loader = build_detection_test_loader(cfg, "beers_valid")
#print("Val: ", val_loader)
inference_on_dataset(trainer.model, val_loader, evaluator)

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode

dataset_dicts = DatasetCatalog.get("beers_valid")
meta_valid = MetadataCatalog.get("beers_valid")

for d in random.sample(dataset_dicts, 15):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1], metadata=meta_valid, scale=0.8, instance_mode=ColorMode.IMAGE_BW)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))


    plt.figure(figsize=(15,15))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show() 

In [ ]:
# Convert video to image frame
!mkdir -p /home/rubio/facebook/detectron2/notebook/output/video2image
!rm  /home/rubio/facebook/detectron2/notebook/output/video2image/*jpg
!ffmpeg  -i "/home/rubio/Downloads/video-30fps.mp4" -r 1 /home/rubio/facebook/detectron2/notebook/output/video2image/image_%03d.jpg #-hide_banner -loglevel error

In [ ]:
import glob
import cv2
import glob

predictor = DefaultPredictor(cfg)

from detectron2.utils.visualizer import ColorMode

for imageName in glob.glob('/home/rubio/facebook/detectron2/notebook/output/video2image/*jpg'):
  img = cv2.imread(imageName)
  outputs = predictor(img)  
  v = Visualizer(img[:, :, ::-1],
                metadata=meta_valid, 
                scale=0.8
                # instance_mode=ColorMode.IMAGE_BW
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  plt.figure(figsize=(15,15))
  plt.imshow(out.get_image()[:, :, ::-1])
  plt.show() 

In [ ]:
# # This is the video we're going to process
from IPython.display import YouTubeVideo, display
# video = YouTubeVideo("ll8TgCZ0plk", width=500)
# display(video)

In [ ]:
with open('model.cfg', 'w+') as f:
    f.write(cfg.dump())
!cat model.cfg

In [ ]:
# Run frame-by-frame inference demo on this video (takes 3-4 minutes) with the "demo.py" tool we provided in the repo.
# !git clone https://github.com/facebookresearch/detectron2
!python3 ../demo/demo.py --config-file model.cfg  --confidence-threshold 0.5 \
 --output video-output2.mkv \
--video-input "/home/rubio/Downloads/WhatsApp Video 2021-01-28 at 12.41.13.mp4" \
 --opts MODEL.WEIGHTS output/model_final.pth 
